In [1]:
import numpy as np

In [2]:
def show(data):
    n, c = data.shape[0], data.shape[-1]
    for i in np.randint(0, n, )

def gdk(n=1000, d=(2,2), k=2):
    is_bw = d[-1] == 3
    data = np.zeros((n, *d))
    
    return data
gdk()

(2, 2) 2
